# Import library

In [829]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Import data

## Green space from landuse key

In [830]:
green_landuse_area = gpd.read_file('../data/raw/osm/green space/green_landuse_area.geojson')

In [831]:
green_landuse_area = green_landuse_area.to_crs(epsg=32633)

In [832]:
green_landuse_area.columns

Index(['element', 'id', 'landuse', 'name', 'area:highway', 'old_name',
       'source', 'addr:city', 'addr:country', 'addr:housenumber',
       'addr:postcode', 'addr:street', 'email', 'website', 'barrier',
       'operator', 'phone', 'sport', 'created_by', 'note', 'opening_hours',
       'wikidata', 'check_date', 'access', 'description', 'leisure',
       'contact:email', 'contact:phone', 'contact:website', 'start_date',
       'disused:leisure', 'leaf_type', 'toilets:wheelchair', 'tourism',
       'wheelchair', 'height', 'surface', 'leaf_cycle', 'comment:history',
       'alt_name', 'kerb', 'produce', 'trees', 'name:signed', 'description:de',
       'embankment', 'building:levels', 'landcover', 'image', 'noname',
       'amenity', 'lit', 'parking', 'smoothness', 'meadow', 'wikipedia',
       'genus:de', 'natural', 'species', 'boundary', 'protect_class',
       'protection_title', 'ref', 'short_protection_title', 'crop', 'fixme',
       'level', 'type', 'name:hsb', 'layer', 'geometry'

In [833]:
green_landuse_area = green_landuse_area[['element', 'id', 'landuse', 
                                         'name', 'barrier', 'note', 
                                         'opening_hours', 'access', 'description', 
                                         'leisure', 'tourism', 'surface', 
                                         'landcover', 'amenity', 'natural', 
                                         'type', 'geometry']]

In [834]:
green_landuse_area['area'] = green_landuse_area['geometry'].area

In [835]:
green_landuse_area['access'].unique()

array([None, 'yes', 'private'], dtype=object)

In [836]:
print('total rows:', green_landuse_area.shape[0])  # number of rows
for i in green_landuse_area['access'].unique():
    if i == None:
        print('access is None:', green_landuse_area[green_landuse_area['access'].isnull()].shape[0])
    else:
        # print number of rows that has access e.g. private,yes,none or others
        print('access is', i , ':', green_landuse_area[green_landuse_area['access']==i].shape[0])

total rows: 2352
access is None: 2340
access is yes : 1
access is private : 11


In [837]:
green_landuse_area['type'].unique()

array(['multipolygon', None], dtype=object)

In [838]:
green_landuse_area['geometry'].type.unique()

array(['MultiPolygon', 'Polygon'], dtype=object)

## Green space from leisure key

In [839]:
green_leisure_area = gpd.read_file('../data/raw/osm/green space/green_leisure_area.geojson')

In [840]:
green_leisure_area = green_leisure_area.to_crs(epsg=32633)

In [841]:
green_leisure_area.columns

Index(['element', 'id', 'access', 'leisure', 'name', 'opening_hours',
       'operator', 'wheelchair', 'sport', 'description', 'layer', 'check_date',
       'playground', 'source', 'lit', 'max_age', 'min_age', 'addr:city',
       'addr:housenumber', 'addr:postcode', 'addr:street', 'note', 'surface',
       'indoor', 'check_date:opening_hours', 'cn_tud:token', 'fee', 'website',
       'garden:type', 'operator:type', 'reservation', 'email', 'old_name',
       'old_name:1897', 'alt_name', 'loc_name', 'wikidata', 'wikipedia',
       'heritage', 'name:etymology:wikidata', 'created_by', 'addr:country',
       'barrier', 'dog', 'playground:theme', 'toilets:wheelchair',
       'contact:email', 'contact:fax', 'contact:phone', 'contact:website',
       'attraction', 'name:cs', 'species:wikidata', 'landuse', 'image', 'ref',
       'phone', 'natural', 'area', 'bicycle', 'service', 'vehicle', 'hoops',
       'height', 'place', 'operator:wikidata', 'start_date', 'species:de',
       'description:de'

In [842]:
green_leisure_area['charge'].unique()

array([None, '8-14€ pro Stunde und Feld'], dtype=object)

In [843]:
green_leisure_area = green_leisure_area[['element', 'id', 'access', 
                                         'leisure', 'name', 'opening_hours', 'indoor',
                                         'operator', 'description', 'playground',
                                         'note', 'surface', 'fee', 
                                         'barrier', 'landuse', 'natural',
                                         'charge', 'landcover', 'fence_type', 
                                         'wall', 'type', 'geometry']]

In [844]:
green_leisure_area['area'] = green_leisure_area['geometry'].area

In [845]:
green_leisure_area['access'].unique()

array(['yes', 'private', None, 'customers', 'permissive', 'permit'],
      dtype=object)

In [846]:
print('total rows:', green_leisure_area.shape[0])  # number of rows
for i in green_leisure_area['access'].unique():
    if i == None:
        print('access is None:', green_leisure_area[green_leisure_area['access'].isnull()].shape[0])
    else:
        # print number of rows that has access e.g. private,yes,none or others
        print('access is', i , ':', green_leisure_area[green_leisure_area['access']==i].shape[0])

total rows: 1136
access is yes : 253
access is private : 138
access is None: 696
access is customers : 27
access is permissive : 19
access is permit : 3


### Separate playground out as we will use it to map PUGS

In [847]:
# filter playgrounds that are indoor
playground_gdf = green_leisure_area[(green_leisure_area['leisure']=='playground') & ~(green_leisure_area['indoor']=='yes')]

In [848]:
green_leisure_area_new = green_leisure_area[(green_leisure_area['leisure']!='playground') | (green_leisure_area['indoor']=='yes')]

## Green space from nature key

In [849]:
green_natural_area = gpd.read_file('../data/raw/osm/green space/green_natural_area.geojson')

In [850]:
green_natural_area = green_natural_area.to_crs(epsg=32633)

In [851]:
green_natural_area.columns

Index(['element', 'id', 'natural', 'attraction', 'name', 'name:cs',
       'species:de', 'species:wikidata', 'wheelchair', 'leaf_type', 'fixme',
       'note', 'leisure', 'leaf_cycle', 'description', 'embankment', 'barrier',
       'landuse', 'source', 'height', 'old_name:cs', 'level', 'man_made',
       'description:de', 'image', 'landcover', 'comment', 'access', 'fee',
       'species:wikipedia', 'type', 'geometry'],
      dtype='object')

In [852]:
green_natural_area['comment'].unique()

array([None,
       'Mitte November 2018 startete eine heftige Baggeraktivität hier'],
      dtype=object)

In [853]:
green_natural_area = green_natural_area[['element', 'id', 'natural', 
                                         'name', 'note', 'leisure',
                                         'description', 'barrier', 'landuse',
                                         'landcover', 'access', 'fee', 'type', 'geometry']]

In [854]:
green_natural_area['area'] = green_natural_area['geometry'].area

In [855]:
green_natural_area['access'].unique()

array([None, 'yes'], dtype=object)

In [856]:
print('total rows:', green_natural_area.shape[0])  # number of rows
for i in green_natural_area['access'].unique():
    if i == None:
        print('access is None:', green_natural_area[green_natural_area['access'].isnull()].shape[0])
    else:
        # print number of rows that has access e.g. private,yes,none or others
        print('access is', i , ':', green_natural_area[green_natural_area['access']==i].shape[0])

total rows: 529
access is None: 526
access is yes : 3


## Green space from camp site

In [857]:
green_campsite_area = gpd.read_file('../data/raw/osm/green space/green_campsite_area.geojson')

In [858]:
green_campsite_area = green_campsite_area.to_crs(epsg=32633)

In [859]:
green_campsite_area.head()

,element,id,tourism,shelter,seats,covered,access,addr:city,addr:housenumber,addr:postcode,addr:street,geometry
0,node,3128780402,picnic_site,None,None,None,None,None,None,None,None,POINT (412264.551 5653901.83)
1,node,3163118403,picnic_site,no,None,None,None,None,None,None,None,POINT (410834.202 5654098.847)
2,node,3806645587,picnic_site,no,None,None,None,None,None,None,None,POINT (410891.184 5653950.093)
3,node,4352882229,picnic_site,None,None,None,None,None,None,None,None,POINT (409894.214 5653944.981)
4,node,4888415416,picnic_site,None,20,None,None,None,None,None,None,POINT (410702.148 5653820.301)


In [860]:
green_campsite_area['area'] = green_campsite_area['geometry'].area

In [861]:
print('total rows:', green_campsite_area.shape[0])  # number of rows
for i in green_campsite_area['access'].unique():
    if i == None:
        print('access is None:', green_campsite_area[green_campsite_area['access'].isnull()].shape[0])
    else:
        # print number of rows that has access e.g. private,yes,none or others
        print('access is', i , ':', green_campsite_area[green_campsite_area['access']==i].shape[0])

total rows: 11
access is None: 9
access is private : 2


## POI/Amenity from amenity 

In [862]:
poi_amenity = gpd.read_file('../data/raw/osm/amenity/poi_amenity.geojson')

In [863]:
poi_amenity = poi_amenity.to_crs(epsg=32633)

In [864]:
poi_amenity.head()

,element,id,amenity,backrest,check_date,material,covered,colour,seats,armrest,...,lit,bench:type,vending,access,operator,deposit_ring,leisure,barrier,man_made,geometry
0,node,255925960,waste_basket,None,NaT,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (409010.616 5654975)
1,node,266687099,bench,yes,NaT,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (408942.691 5654898.334)
2,node,304869266,bench,yes,2021-08-13,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (412089.991 5656475.039)
3,node,304869267,bench,yes,2021-08-13,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (412091.465 5656481.777)
4,node,304869268,bench,yes,2021-08-13,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (412090.847 5656487.85)


## POI/Amenity from leisure

In [865]:
poi_leisure = gpd.read_file('../data/raw/osm/amenity/poi_leisure.geojson')

In [866]:
poi_leisure = poi_leisure.to_crs(epsg=32633)

In [867]:
poi_leisure.head()

,element,id,access,leisure,name,opening_hours,operator,wheelchair,sport,layer,...,capacity,fence_type,fixme,addr:country,contact:email,contact:mobile,contact:website,level,type,geometry
0,node,266593887,yes,playground,Würzburger Straße - Park,Mo-Su 08:00-22:00,Landeshauptstadt Dresden,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (409905.579 5653957.552)
1,node,274970363,private,playground,None,None,Vonovia,yes,None,None,...,None,None,None,None,None,None,None,None,None,POINT (410248.323 5654411.132)
2,node,280462694,yes,playground,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (413613.876 5655700.3)
3,node,289488035,yes,playground,Spielplatz Schanzenstraße,None,None,yes,table_tennis,None,...,None,None,None,None,None,None,None,None,None,POINT (412445.204 5659013.112)
4,node,372545516,customers,playground,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (414579.736 5654048.389)


## Road network

In [868]:
road_network_edges = gpd.read_file('../data/raw/osm/network/road network/road_network_edges.geojson')
road_network_nodes = gpd.read_file('../data/raw/osm/network/road network/road_network_nodes.geojson')

Skipping field highway: unsupported OGR type: 5
Skipping field lanes: unsupported OGR type: 5
Skipping field maxspeed: unsupported OGR type: 5
Skipping field name: unsupported OGR type: 5
Skipping field width: unsupported OGR type: 5


## Cycle network

In [869]:
cycle_network_edges = gpd.read_file('../data/raw/osm/network/cycle network/cycle_network_edges.geojson')
cycle_network_nodes = gpd.read_file('../data/raw/osm/network/cycle network/cycle_network_nodes.geojson')

Skipping field highway: unsupported OGR type: 5
Skipping field lanes: unsupported OGR type: 5
Skipping field maxspeed: unsupported OGR type: 5
Skipping field name: unsupported OGR type: 5
Skipping field ref: unsupported OGR type: 5
Skipping field access: unsupported OGR type: 5
Skipping field width: unsupported OGR type: 5
Skipping field service: unsupported OGR type: 5
Skipping field tunnel: unsupported OGR type: 5


## Footpath network

In [870]:
footpath_network_edges = gpd.read_file('../data/raw/osm/network/footpath network/footpath_network_edges.geojson')
footpath_network_nodes = gpd.read_file('../data/raw/osm/network/footpath network/footpath_network_nodes.geojson')

Skipping field highway: unsupported OGR type: 5
Skipping field lanes: unsupported OGR type: 5
Skipping field maxspeed: unsupported OGR type: 5
Skipping field name: unsupported OGR type: 5
Skipping field ref: unsupported OGR type: 5
Skipping field access: unsupported OGR type: 5
Skipping field bridge: unsupported OGR type: 5
Skipping field width: unsupported OGR type: 5
Skipping field service: unsupported OGR type: 5
Skipping field tunnel: unsupported OGR type: 5


# Spatial join green space polygons (within)
If small polygon is in larger polygon, only keep the larger polygon

In [871]:
gdf_list = [green_landuse_area, green_leisure_area_new, green_natural_area, green_campsite_area]
count = 0
for i in range(len(gdf_list)):
    for j in range(len(gdf_list)):
        # right contain left or not
        temp_gdf = gdf_list[i].sjoin(gdf_list[j], how='right', predicate='within')
        print('i:', i, 'j:', j, 'row:', temp_gdf.shape[0])
        if count == 0:
            joined_green_space = temp_gdf
        else:
            joined_green_space = pd.concat([joined_green_space, temp_gdf], ignore_index=True)
        count += 1

i: 0 j: 0 row: 2358
i: 0 j: 1 row: 844
i: 0 j: 2 row: 529
i: 0 j: 3 row: 11
i: 1 j: 0 row: 2383
i: 1 j: 1 row: 795
i: 1 j: 2 row: 529
i: 1 j: 3 row: 11
i: 2 j: 0 row: 2371
i: 2 j: 1 row: 733
i: 2 j: 2 row: 530
i: 2 j: 3 row: 11
i: 3 j: 0 row: 2352
i: 3 j: 1 row: 723
i: 3 j: 2 row: 529
i: 3 j: 3 row: 11


In [872]:
joined_green_space['id_left'] = joined_green_space['id_left'].fillna(-999)
joined_green_space['id_left'] = joined_green_space['id_left'].astype(int).astype(str)
joined_green_space['id_right'] = joined_green_space['id_right'].astype(str)

In [873]:
joined_green_space[joined_green_space['id_right'] == '365345'][['id_left', 'id_right', 'area_left', 'area_right']]

,id_left,id_right,area_left,area_right
2468,157942,365345,94969.782463,1.517605e+06
2469,157943,365345,100603.258852,1.517605e+06
2470,157944,365345,285535.962086,1.517605e+06
2471,396375,365345,1493.478101,1.517605e+06
2472,421481,365345,181451.349952,1.517605e+06
...,...,...,...,...
9942,28889290,365345,150.398311,1.517605e+06
9943,28889293,365345,49.059637,1.517605e+06
9944,28889296,365345,381.918061,1.517605e+06
9945,28889298,365345,71.053700,1.517605e+06


In [874]:
joined_green_space.shape[0]

14720

## Remove itself within itself row

In [913]:
joined_green_space = joined_green_space[joined_green_space['id_left'] != joined_green_space['id_right']]

In [914]:
joined_green_space.shape[0]

11095

In [915]:
joined_green_space[joined_green_space['id_right'] == '365345'][['id_left', 'id_right', 'area_left', 'area_right']]['id_left'].unique()

array(['157942', '157943', '157944', '396375', '421481', '36006691',
       '36006694', '36006695', '36006696', '36006697', '36006698',
       '36006699', '36006700', '36006701', '36006702', '36006703',
       '36006704', '36006705', '36006706', '36006707', '36006708',
       '36006709', '36006710', '36006711', '36006712', '36006713',
       '36006714', '36006715', '36006716', '36006720', '36006721',
       '36006722', '36006723', '36006724', '36006725', '36006726',
       '36006728', '36006729', '36006732', '38867041', '48991210',
       '48991211', '48991217', '48991221', '48991222', '48991223',
       '48992703', '48992704', '48992705', '48992706', '48993727',
       '48994214', '48994220', '48994225', '48994228', '48994231',
       '48994239', '48994242', '389227741', '424056390', '424202562',
       '424202563', '424202564', '424202566', '424202568', '424202569',
       '424202570', '424202571', '426633229', '426633231', '776328519',
       '776328522', '1312793920', '1312793921',

In [916]:
clean_joined_green_space = joined_green_space[~joined_green_space['id_right'].isin(joined_green_space['id_left'])]

In [917]:
clean_joined_green_space

,index_left,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,...,addr:housenumber_left,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right
18,1455.0,way,709615762,meadow,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,1058.0,way,345110108,grass,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,1080.0,way,352739233,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,1942.0,way,928898643,meadow,Glatthaferwiese am Elbufer Johannstadt,None,None,None,None,Artenreiche Glatthaferwiese mit hohem floristi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,1404.0,way,578835840,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14704,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14705,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14706,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14707,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [918]:
# 157942 is small polygon inside
clean_joined_green_space[clean_joined_green_space['id_right'] == '157942'][['id_left', 'id_right', 'area_left', 'area_right']]

,id_left,id_right,area_left,area_right


In [919]:
clean_joined_green_space[clean_joined_green_space['id_right'] == '365345']

,index_left,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,...,addr:housenumber_left,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right
2468,1.0,relation,157942,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,2.0,relation,157943,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470,3.0,relation,157944,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2471,4.0,relation,396375,grass,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2472,5.0,relation,421481,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,32.0,way,28889290,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9943,33.0,way,28889293,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9944,34.0,way,28889296,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9945,35.0,way,28889298,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [920]:
clean_joined_green_space[clean_joined_green_space['id_right'] == '365345'].columns

Index(['index_left', 'element_left', 'id_left', 'landuse_left', 'name_left',
       'barrier_left', 'note_left', 'opening_hours_left', 'access_left',
       'description_left', 'leisure_left', 'tourism_left', 'surface_left',
       'landcover_left', 'amenity_left', 'natural_left', 'type_left',
       'area_left', 'element_right', 'id_right', 'landuse_right', 'name_right',
       'barrier_right', 'note_right', 'opening_hours_right', 'access_right',
       'description_right', 'leisure_right', 'tourism_right', 'surface_right',
       'landcover_right', 'amenity_right', 'natural_right', 'type_right',
       'geometry', 'area_right', 'tourism', 'amenity', 'indoor', 'operator',
       'playground', 'fee', 'charge', 'fence_type', 'wall', 'opening_hours',
       'surface', 'landuse', 'name', 'barrier', 'note', 'description',
       'leisure', 'landcover', 'natural', 'type', 'shelter', 'seats',
       'covered', 'addr:city', 'addr:housenumber', 'addr:postcode',
       'addr:street', 'indoor_le

In [921]:
clean_joined_green_space[(clean_joined_green_space['id_right'] == '365345')][['id_left', 'id_right', 'area_left', 'area_right', 'name_right', 'element_right']]

,id_left,id_right,area_left,area_right,name_right,element_right
2468,157942,365345,94969.782463,1.517605e+06,Großer Garten,relation
2469,157943,365345,100603.258852,1.517605e+06,Großer Garten,relation
2470,157944,365345,285535.962086,1.517605e+06,Großer Garten,relation
2471,396375,365345,1493.478101,1.517605e+06,Großer Garten,relation
2472,421481,365345,181451.349952,1.517605e+06,Großer Garten,relation
...,...,...,...,...,...,...
9942,28889290,365345,150.398311,1.517605e+06,Großer Garten,relation
9943,28889293,365345,49.059637,1.517605e+06,Großer Garten,relation
9944,28889296,365345,381.918061,1.517605e+06,Großer Garten,relation
9945,28889298,365345,71.053700,1.517605e+06,Großer Garten,relation


In [922]:
clean_joined_green_space['id_right'].unique()

array(['3807857', '238260545', '239351758', ..., '30642601', '431068944',
       '431069414'], dtype=object)

In [923]:
clean_joined_green_space[(clean_joined_green_space['id_right'] == '1340028517')][['id_left', 'id_right', 'area_left', 'area_right', 'name_right', 'element_right']]

,id_left,id_right,area_left,area_right,name_right,element_right
6122,3645255648,1340028517,0.0,8049.696317,None,way
9828,-999,1340028517,NaN,8049.696317,None,way
13454,-999,1340028517,NaN,8049.696317,NaN,way


In [924]:
clean_joined_green_space[clean_joined_green_space['id_left'] == '359896413']

,index_left,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,...,addr:housenumber_left,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right


In [925]:
# green_leisure_area[green_leisure_area['id'] == 359896413].explore()

In [926]:
# clean_joined_green_space[clean_joined_green_space['id_right'] == '1340028517'].explore()

In [927]:
clean_joined_green_space = clean_joined_green_space.drop_duplicates(subset=['id_right', 'id_left'])

In [928]:
clean_joined_green_space[(clean_joined_green_space['id_right'] == '1340028517')][['id_left', 'id_right', 'area_left', 'area_right', 'name_right', 'element_right']]

,id_left,id_right,area_left,area_right,name_right,element_right
6122,3645255648,1340028517,0.0,8049.696317,None,way
9828,-999,1340028517,NaN,8049.696317,None,way


In [929]:
clean_joined_green_space

,index_left,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,...,addr:housenumber_left,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right
18,1455.0,way,709615762,meadow,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,1058.0,way,345110108,grass,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,1080.0,way,352739233,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,1942.0,way,928898643,meadow,Glatthaferwiese am Elbufer Johannstadt,None,None,None,None,Artenreiche Glatthaferwiese mit hohem floristi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,1404.0,way,578835840,forest,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13598,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13615,3.0,node,4352882229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13879,NaN,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filter indoor space out

In [930]:
clean_joined_green_space = clean_joined_green_space[clean_joined_green_space['indoor'] != 'yes']

## Clean columns

In [931]:
clean_joined_green_space.columns

Index(['index_left', 'element_left', 'id_left', 'landuse_left', 'name_left',
       'barrier_left', 'note_left', 'opening_hours_left', 'access_left',
       'description_left', 'leisure_left', 'tourism_left', 'surface_left',
       'landcover_left', 'amenity_left', 'natural_left', 'type_left',
       'area_left', 'element_right', 'id_right', 'landuse_right', 'name_right',
       'barrier_right', 'note_right', 'opening_hours_right', 'access_right',
       'description_right', 'leisure_right', 'tourism_right', 'surface_right',
       'landcover_right', 'amenity_right', 'natural_right', 'type_right',
       'geometry', 'area_right', 'tourism', 'amenity', 'indoor', 'operator',
       'playground', 'fee', 'charge', 'fence_type', 'wall', 'opening_hours',
       'surface', 'landuse', 'name', 'barrier', 'note', 'description',
       'leisure', 'landcover', 'natural', 'type', 'shelter', 'seats',
       'covered', 'addr:city', 'addr:housenumber', 'addr:postcode',
       'addr:street', 'indoor_le

In [932]:
clean_joined_green_space = clean_joined_green_space.drop(columns=['index_left', 'shelter', 'seats', 
                                                                  'covered', 'indoor', 'area_left',
                                                                  'addr:city', 'addr:housenumber', 
                                                                  'addr:postcode', 'addr:street'])

In [933]:
clean_joined_green_space['barrier_right'].unique()

array(['fence', None, 'wall', nan], dtype=object)

# Classify public and private green spaces

## Filter out the polygon that area < 100 m2 (10m x 10m)

In [934]:
clean_joined_green_space.shape[0]

3616

In [935]:
clean_joined_green_space = clean_joined_green_space[clean_joined_green_space['area_right'] > 100]

In [936]:
clean_joined_green_space.shape[0]

2753

## 1. Use fee and charge tag to classify private area

In [937]:
clean_joined_green_space.loc[(clean_joined_green_space['fee_left'] == 'yes')|
                             (clean_joined_green_space['fee_right'] == 'yes')|
                             (clean_joined_green_space['charge'].isna() == False)|
                             (clean_joined_green_space['charge_left'].isna() == False)|
                             (clean_joined_green_space['charge_right'].isna() == False), 'is_public'] = 'no'

In [938]:
clean_joined_green_space[clean_joined_green_space['is_public'] == 'no']

,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,leisure_left,...,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right,is_public
2864,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
2946,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
2950,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
3117,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
3118,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no


## 2. Use access tag to classify
- Access = yes/permissive -> public
- Access = no/private/pcustomers/restricted -> private

Note: Consider both *_right* and *_left* column (Assumption is if the access of small polygon is yes, the access of larger polygon is yes as well)

In [939]:
clean_joined_green_space[((clean_joined_green_space['access_right'] == 'yes')|(clean_joined_green_space['access_left'] == 'yes')) & (clean_joined_green_space['is_public'] != 'no')].explore()

In [940]:
clean_joined_green_space.loc[((clean_joined_green_space['access_right'] == 'yes')|
                              (clean_joined_green_space['access_left'] == 'yes')|
                              (clean_joined_green_space['access_right'] == 'permissive')|
                              (clean_joined_green_space['access_left'] == 'permissive')) & 
                              (clean_joined_green_space['is_public'] != 'no'), 'is_public'] = 'yes'

In [942]:
clean_joined_green_space[clean_joined_green_space['is_public'].isna()]

,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,leisure_left,...,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right,is_public
18,way,709615762,meadow,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,way,345110108,grass,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,way,352739233,forest,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,way,928898643,meadow,Glatthaferwiese am Elbufer Johannstadt,None,None,None,None,Artenreiche Glatthaferwiese mit hohem floristi...,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,way,578835840,forest,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13598,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13615,node,4352882229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13879,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [943]:
access_tag = clean_joined_green_space['access_right'].unique().tolist()
for i in clean_joined_green_space['access_left'].unique().tolist():
    if i not in access_tag:
        access_tag.append(i)

In [944]:
access_tag = [i for i in access_tag if (str(i) != 'nan')&(i is not None)]

In [945]:
access_tag

['private', 'yes', 'permissive', 'customers', 'permit']

In [946]:
private_tag = list(access_tag)
private_tag.remove('yes')
private_tag.remove('permissive')
private_tag

['private', 'customers', 'permit']

In [947]:
clean_joined_green_space[(clean_joined_green_space['access_right'].isin(private_tag)|clean_joined_green_space['access_left'].isin(private_tag))& clean_joined_green_space['is_public'].isna()].explore()

In [949]:
clean_joined_green_space[(clean_joined_green_space['access_right'].isin(private_tag)|clean_joined_green_space['access_left'].isin(private_tag))& clean_joined_green_space['is_public'].isna()].shape[0]

46

In [950]:
clean_joined_green_space.loc[(clean_joined_green_space['access_right'].isin(private_tag)|
                              clean_joined_green_space['access_left'].isin(private_tag))& 
                              clean_joined_green_space['is_public'].isna(), 'is_public'] = 'no'

In [951]:
clean_joined_green_space[clean_joined_green_space['is_public'].isna()]

,element_left,id_left,landuse_left,name_left,barrier_left,note_left,opening_hours_left,access_left,description_left,leisure_left,...,addr:postcode_left,addr:street_left,shelter_right,seats_right,covered_right,addr:city_right,addr:housenumber_right,addr:postcode_right,addr:street_right,is_public
18,way,709615762,meadow,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
685,way,345110108,grass,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,way,352739233,forest,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,way,928898643,meadow,Glatthaferwiese am Elbufer Johannstadt,None,None,None,None,Artenreiche Glatthaferwiese mit hohem floristi...,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,way,578835840,forest,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13598,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13615,node,4352882229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13879,NaN,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Use leisure=park tag to classify public area

In [954]:
clean_joined_green_space[(clean_joined_green_space['leisure_right'] == 'park')&(clean_joined_green_space['is_public'].isna())].explore()

In [955]:
clean_joined_green_space.loc[(clean_joined_green_space['leisure_right'] == 'park')&(clean_joined_green_space['is_public'].isna()), 'is_public'] = 'yes'

In [956]:
clean_joined_green_space[clean_joined_green_space['is_public'].isna()].shape[0]

2283

## 4. Use barrier tag to classify private area

Need to confirm that **allotments** is public or private? (Maybe in case of Germany/Dresden first)

In [962]:
clean_joined_green_space.columns

Index(['element_left', 'id_left', 'landuse_left', 'name_left', 'barrier_left',
       'note_left', 'opening_hours_left', 'access_left', 'description_left',
       'leisure_left', 'tourism_left', 'surface_left', 'landcover_left',
       'amenity_left', 'natural_left', 'type_left', 'element_right',
       'id_right', 'landuse_right', 'name_right', 'barrier_right',
       'note_right', 'opening_hours_right', 'access_right',
       'description_right', 'leisure_right', 'tourism_right', 'surface_right',
       'landcover_right', 'amenity_right', 'natural_right', 'type_right',
       'geometry', 'area_right', 'tourism', 'amenity', 'operator',
       'playground', 'fee', 'charge', 'fence_type', 'wall', 'opening_hours',
       'surface', 'landuse', 'name', 'barrier', 'note', 'description',
       'leisure', 'landcover', 'natural', 'type', 'indoor_left',
       'operator_left', 'playground_left', 'fee_left', 'charge_left',
       'fence_type_left', 'wall_left', 'indoor_right', 'operator_right',

In [963]:
print(clean_joined_green_space['barrier_right'].unique())
print(clean_joined_green_space['barrier_left'].unique())
print(clean_joined_green_space['barrier'].unique())
print('-'*20)
print(clean_joined_green_space['fence_type_right'].unique())
print(clean_joined_green_space['fence_type_left'].unique())
print(clean_joined_green_space['fence_type'].unique())

['fence' None 'wall' nan]
[None 'hedge' nan 'fence']
[nan None]
--------------------
[nan]
[nan]
[nan]


In [969]:
# clean_joined_green_space[((clean_joined_green_space['barrier_right'] == 'fence')|(clean_joined_green_space['barrier_right'] == 'wall'))&(clean_joined_green_space['is_public'].isna())].explore()

In [971]:
clean_joined_green_space[(~clean_joined_green_space['barrier_right'].isna())&(clean_joined_green_space['is_public'].isna())].shape[0]

36

In [972]:
clean_joined_green_space.loc[(~clean_joined_green_space['barrier_right'].isna())&(clean_joined_green_space['is_public'].isna()), 'is_public'] = 'no'

In [973]:
clean_joined_green_space[clean_joined_green_space['is_public'].isna()].shape[0]

2247

## Calculate additional information to classify public/private area